На этой неделе мы познакомимся с популярной библиотекой Vowpal Wabbit и попробуем ее на данных по посещению сайтов.

#### План 6 недели:

- Часть 1. Статья по Vowpal Wabbit
- Часть 2. Применение Vowpal Wabbit к данным по посещению сайтов
- 2.1. Подготовка данных
- 2.2. Валидация по отложенной выборке
- 2.3. Валидация по тестовой выборке (Public Leaderboard)

В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":

Стохатический градиентный спуск
Линейные модели. sklearn.linear_model. Классификация
Также будет полезна презентация лектора специализации Евгения Соколова. И, конечно же, документация Vowpal Wabbit.

### Задание
- Заполните код в этой тетрадке
- Если вы проходите специализацию Яндеса и МФТИ, пошлите файл с ответами в соответствующем Programming Assignment.
- Если вы проходите курс ODS, выберите ответы в веб-форме.

#### Часть 1. Статья про Vowpal Wabbit
Прочитайте статью про Vowpal Wabbit на Хабре из серии открытого курса OpenDataScience по машинному обучению. Материал для этой статьи зародился из нашей специализации. Скачайте тетрадку, прилагаемую к статье, посмотрите код, изучите его, поменяйте, только так можно разобраться с Vowpal Wabbit.

#### Часть 2. Применение Vowpal Wabbit к данным по посещению сайтов
2.1. Подготовка данных
Далее посмотрим на Vowpal Wabbit в деле. Правда, в задаче нашего соревнования при бинарной классификации веб-сессий мы разницы не заметим – как по качеству, так и по скорости работы (хотя можете проверить), продемонстрируем всю резвость VW в задаче классификации на 400 классов. Исходные данные все те же самые, но выделено 400 пользователей, и решается задача их идентификации. Скачайте данные отсюда – файлы train_sessions_400users.csv и test_sessions_400users.csv.

In [1]:
import os
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [2]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = 'capstone_user_identification'

Загрузим обучающую и тестовую выборки. Можете заметить, что тестовые сессии здесь по времени четко отделены от сессий в обучающей выборке.

In [3]:
train_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'train_sessions_400users.csv'), 
                           index_col='session_id')

In [4]:
test_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'test_sessions_400users.csv'), 
                           index_col='session_id')

In [5]:
train_df_400.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,user_id
session_id,,,,,,,,,,,,,,,,,,,,,
1,23713,2014-03-24 15:22:40,23720.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:54,23720.0,2014-03-24 15:22:54,...,2014-03-24 15:22:55,23713.0,2014-03-24 15:23:01,23713.0,2014-03-24 15:23:03,23713.0,2014-03-24 15:23:04,23713.0,2014-03-24 15:23:05,653
2,8726,2014-04-17 14:25:58,8725.0,2014-04-17 14:25:59,665.0,2014-04-17 14:25:59,8727.0,2014-04-17 14:25:59,45.0,2014-04-17 14:25:59,...,2014-04-17 14:26:01,45.0,2014-04-17 14:26:01,5320.0,2014-04-17 14:26:18,5320.0,2014-04-17 14:26:47,5320.0,2014-04-17 14:26:48,198
3,303,2014-03-21 10:12:24,19.0,2014-03-21 10:12:36,303.0,2014-03-21 10:12:54,303.0,2014-03-21 10:13:01,303.0,2014-03-21 10:13:24,...,2014-03-21 10:13:36,303.0,2014-03-21 10:13:54,309.0,2014-03-21 10:14:01,303.0,2014-03-21 10:14:06,303.0,2014-03-21 10:14:24,34
4,1359,2013-12-13 09:52:28,925.0,2013-12-13 09:54:34,1240.0,2013-12-13 09:54:34,1360.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:54:34,...,2013-12-13 09:54:34,1346.0,2013-12-13 09:54:34,1345.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:58:19,1345.0,2013-12-13 09:58:19,601
5,11,2013-11-26 12:35:29,85.0,2013-11-26 12:35:31,52.0,2013-11-26 12:35:31,85.0,2013-11-26 12:35:32,11.0,2013-11-26 12:35:32,...,2013-11-26 12:35:32,11.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:03,10.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:04,273


Видим, что в обучающей выборке 182793 сессий, в тестовой – 46473, и сессии действительно принадлежат 400 различным пользователям.

In [6]:
train_df_400.shape, test_df_400.shape, train_df_400['user_id'].nunique()

((182793, 21), (46473, 20), 400)

Vowpal Wabbit любит, чтоб метки классов были распределены от 1 до K, где K – число классов в задаче классификации (в нашем случае – 400). Поэтому придется применить LabelEncoder, да еще и +1 потом добавить (LabelEncoder переводит метки в диапозон от 0 до K-1). Потом надо будет применить обратное преобразование.

In [7]:
y = train_df_400['user_id']
class_encoder = LabelEncoder()
y_for_vw = class_encoder.fit_transform(y) + 1

### Далее будем сравнивать VW с SGDClassifier и с логистической регрессией. Всем моделям этим нужна предобработка входных данных. Подготовьте для sklearn-моделей разреженные матрицы, как мы это делали в 5 части:

- объедините обучающиую и тестовую выборки
- выберите только сайты (признаки от 'site1' до 'site10')
- замените пропуски на нули (сайты у нас нумеровались с 0)
- переведите в разреженный формат csr_matrix
- разбейте обратно на обучающую и тестовую части

In [8]:
sites = ['site' + str(i) for i in range(1, 11)]
train_test_df = pd.concat([train_df_400, test_df_400], axis=0)
train_test_df_sites = train_test_df[sites].fillna(0)

In [9]:
def csr_matrix_parameters(X):
    """Calculating parameters for creating a sparse matrix"""
    indptr = [0]
    indices = []
    data = []
    vocabulary = {}
    for session in X:
        for site in session:
            index = vocabulary.setdefault(site, site)
            indices.append(index)
            data.append(1)
        indptr.append(len(indices))
    return [data, indices, indptr]

In [10]:
data, indices, indptr = csr_matrix_parameters(train_test_df_sites.values)
train_test_sparse = csr_matrix((data, indices, indptr))[:, 1:]
X_train_sparse = train_test_sparse[:train_df_400.shape[0], :]
X_test_sparse = train_test_sparse[train_df_400.shape[0]:, :]

### 2.2. Валидация по отложенной выборке

Выделим обучающую (70%) и отложенную (30%) части исходной обучающей выборки. Данные не перемешиваем, учитываем, что сессии отсортированы по времени.

In [11]:
train_share = int(.7 * train_df_400.shape[0])
train_df_part = train_df_400[sites].iloc[:train_share, :]
valid_df = train_df_400[sites].iloc[train_share:, :]
X_train_part_sparse = X_train_sparse[:train_share, :]
X_valid_sparse = X_train_sparse[train_share:, :]

In [12]:
y_train_part = y[:train_share]
y_valid = y[train_share:]
y_train_part_for_vw = y_for_vw[:train_share]
y_valid_for_vw = y_for_vw[train_share:]


Реализуйте функцию, arrays_to_vw, переводящую обучающую выборку в формат Vowpal Wabbit.

Вход:

- X – матрица NumPy (обучающая выборка)
- y (необяз.) - вектор ответов (NumPy). Необязателен, поскольку тестовую матрицу будем обрабатывать этой же функцией
- train – флаг, True в случае обучающей выборки, False – в случае тестовой выборки
- out_file – путь к файлу .vw, в который будет произведена запись

Детали:

- надо пройтись по всем строкам матрицы X и записать через пробел все значения, предварительно добавив вперед нужную метку класса из вектора y и знак-разделитель |
- в тестовой выборке на месте меток целевого класса можно писать произвольные, допустим, 1

In [13]:
def arrays_to_vw(X, y=None, train=True, out_file='tmp.vw'):
    if train == False:
        y = [1] * X.shape[0]
    with open(os.path.join(PATH_TO_DATA, out_file), 'w') as f:
        for i, line in enumerate(X.values):
            f.write(str(y[i]) + ' | ' + ' '.join(map(str, line)) + '\n')

Примените написанную функцию к части обучащей выборки (train_df_part, y_train_part_for_vw), к отложенной выборке (valid_df, y_valid_for_vw), ко всей обучающей выборке и ко всей тестовой выборке. Обратите внимание, что на вход наш метод принимает именно матрицы и вектора NumPy.

In [14]:
%%time
arrays_to_vw(train_df_part, y_train_part_for_vw, 
             out_file='train_part.vw')
arrays_to_vw(valid_df, y_valid_for_vw, 
             out_file='valid.vw')
arrays_to_vw(train_df_400[sites], y_for_vw, 
             out_file='train.vw')
arrays_to_vw(test_df_400[sites], train=False, 
             out_file='test.vw')

Wall time: 3.32 s


Результат должен получиться таким.

In [15]:
!head -3  $PATH_TO_DATA/train_part.vw

262 | 23713.0 23720.0 23713.0 23713.0 23720.0 23713.0 23713.0 23713.0 23713.0 23713.0
82 | 8726.0 8725.0 665.0 8727.0 45.0 8725.0 45.0 5320.0 5320.0 5320.0
16 | 303.0 19.0 303.0 303.0 303.0 303.0 303.0 309.0 303.0 303.0


In [16]:
!head -3  $PATH_TO_DATA/valid.vw

4 | 7.0 923.0 923.0 923.0 11.0 924.0 7.0 924.0 838.0 7.0
160 | 91.0 198.0 11.0 11.0 302.0 91.0 668.0 311.0 310.0 91.0
312 | 27085.0 848.0 118.0 118.0 118.0 118.0 11.0 118.0 118.0 118.0


In [17]:
!head -3 $PATH_TO_DATA/test.vw

1 | 9.0 304.0 308.0 307.0 91.0 308.0 312.0 300.0 305.0 309.0
1 | 838.0 504.0 68.0 11.0 838.0 11.0 838.0 886.0 27.0 305.0
1 | 190.0 192.0 8.0 189.0 191.0 189.0 190.0 2375.0 192.0 8.0


Получившиеся файлы.

In [18]:
!head -3 $PATH_TO_DATA/train_part.vw

262 | 23713.0 23720.0 23713.0 23713.0 23720.0 23713.0 23713.0 23713.0 23713.0 23713.0
82 | 8726.0 8725.0 665.0 8727.0 45.0 8725.0 45.0 5320.0 5320.0 5320.0
16 | 303.0 19.0 303.0 303.0 303.0 303.0 303.0 309.0 303.0 303.0


In [19]:
!head -3  $PATH_TO_DATA/valid.vw

4 | 7.0 923.0 923.0 923.0 11.0 924.0 7.0 924.0 838.0 7.0
160 | 91.0 198.0 11.0 11.0 302.0 91.0 668.0 311.0 310.0 91.0
312 | 27085.0 848.0 118.0 118.0 118.0 118.0 11.0 118.0 118.0 118.0


In [20]:
!head -3 $PATH_TO_DATA/test.vw

1 | 9.0 304.0 308.0 307.0 91.0 308.0 312.0 300.0 305.0 309.0
1 | 838.0 504.0 68.0 11.0 838.0 11.0 838.0 886.0 27.0 305.0
1 | 190.0 192.0 8.0 189.0 191.0 189.0 190.0 2375.0 192.0 8.0


Обучите модель Vowpal Wabbitна выборке train_part.vw. Укажите, что решается задача классификации с 400 классами (--oaa), сделайте 3 прохода по выборке (--passes). Задайте некоторый кэш-файл (--cache_file, можно просто указать флаг -c), так VW будет быстрее делать все следующие после первого проходы по выборке (прошлый кэш-файл удаляется с помощью аргумента -k). Также укажите значение параметра b=26. Это число бит, используемых для хэширования, в данном случае нужно больше, чем 18 по умолчанию. Наконец, укажите random_seed=17. Остальные параметры пока не меняйте, далее уже в свободном режиме соревнования можете попробовать другие функции потерь.

In [21]:
train_part_vw = os.path.join(PATH_TO_DATA, 'train_part.vw')
valid_vw = os.path.join(PATH_TO_DATA, 'valid.vw')
train_vw = os.path.join(PATH_TO_DATA, 'train.vw')
test_vw = os.path.join(PATH_TO_DATA, 'test.vw')
model = os.path.join(PATH_TO_DATA, 'vw_model.vw')
pred = os.path.join(PATH_TO_DATA, 'vw_pred.csv')

In [1]:
from vowpalwabbit import pyvw

ModuleNotFoundError: No module named 'vowpalwabbit'

In [22]:
%%time
!vw -d 'capstone_user_identification/train_part.vw' \
    --oaa 400 --passes 3 -k -c -b 26 --random_seed 17 \
    -f 'capstone_user_identification/vw_model1.vw'

Wall time: 54.2 ms


"vw" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


Запишите прогнозы на выборке valid.vw в vw_valid_pred.csv.

In [28]:
%%time
!vw -i 'capstone_user_identification/vw_model1.vw' -t \
    -d 'capstone_user_identification/valid.vw' \
    -p 'capstone_user_identification/vw_pred1.csv'

Wall time: 54.4 ms


"vw" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


Считайте прогнозы kaggle_data/vw_valid_pred.csv из файла и посмотрите на долю правильных ответов на отложенной части.

In [29]:
vw_valid_pred = pd.read_csv(os.path.join(PATH_TO_DATA, 'vw_pred1.csv'), 
                            header=None)
print('Vompal-Wabbit valid predictions: ' + \
      str(round(accuracy_score(y_valid_for_vw, vw_valid_pred), 3)))

FileNotFoundError: [Errno 2] No such file or directory: 'capstone_user_identification\\vw_pred1.csv'

Теперь обучите SGDClassifier (3 прохода по выборке, логистическая функция потерь) и LogisticRegression на 70% разреженной обучающей выборки – (X_train_part_sparse, y_train_part), сделайте прогноз для отложенной выборки (X_valid_sparse, y_valid) и посчитайте доли верных ответов. Логистическая регрессия будет обучаться не быстро (у меня – 4 минуты) – это нормально. Укажите везде random_state=17, n_jobs=-1. Для SGDClassifier также укажите max_iter=3.

In [30]:
logit = LogisticRegression(solver='liblinear', random_state=17)
sgd_logit = SGDClassifier(loss='log', max_iter=3, random_state=17)

In [ ]:
%%time
logit.fit(X_train_part_sparse, y_train_part)

In [ ]:
%%time
sgd_logit.fit(X_train_part_sparse, y_train_part)

Посчитайте долю правильных ответов на отложенной выборке для Vowpal Wabbit, округлите до 3 знаков после запятой.

Посчитайте долю правильных ответов на отложенной выборке для SGD, округлите до 3 знаков после запятой.

Посчитайте долю правильных ответов на отложенной выборке для логистической регрессии, округлите до 3 знаков после запятой.

In [ ]:
vw_valid_acc = round(accuracy_score(y_valid_for_vw, vw_valid_pred), 3)
sgd_valid_acc = round(accuracy_score(y_valid, sgd_logit.predict(X_valid_sparse)), 3)
logit_valid_acc = round(accuracy_score(y_valid, logit.predict(X_valid_sparse)), 3)

In [ ]:
def write_answer_to_file(answer, file_address):
    with open(file_address, 'w') as out_f:
        out_f.write(str(answer))

In [ ]:
write_answer_to_file(round(vw_valid_acc, 3), 'answers/week6/answer6_1.txt')
write_answer_to_file(round(sgd_valid_acc, 3), 'answers/week6/answer6_2.txt')
write_answer_to_file(round(logit_valid_acc, 3), 'answers/week6/answer6_3.txt')

### 2.3. Валидация по тестовой выборке (Public Leaderboard)¶
##### Обучите модель VW с теми же параметрами на всей обучающей выборке – train.vw.

In [ ]:
%%time
!vw -d 'capstone_user_identification/train.vw' \
    --oaa 400 --passes 3 -k -c -b 26 --random_seed 17 \
    -f 'capstone_user_identification/vw_model2.vw'

Сделайте прогноз для тестовой выборки.

In [ ]:
%%time
!vw -i 'capstone_user_identification/vw_model2.vw' -t \
    -d 'capstone_user_identification/test.vw' \
    -p 'capstone_user_identification/vw_pred2.csv'

Запишите прогноз в файл, примените обратное преобразование меток (был LabelEncoder и потом +1 в меткам) и отправьте решение на Kaggle.

In [ ]:
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
vw_pred = pd.read_csv(os.path.join(PATH_TO_DATA, 'vw_pred2.csv'), 
                            header=None) - 1
vw_pred = class_encoder.inverse_transform(vw_pred)

In [ ]:
write_to_submission_file(vw_pred, os.path.join(PATH_TO_DATA, 'vw_400_users.csv'))

Сделайте то же самое для SGD и логистической регрессии. Тут уже ждать обучение логистической регрессии совсем скучно (заново запускать тетрадку вам не захочется), но давайте дождемся.

In [ ]:
logit2 = LogisticRegression(solver='liblinear', random_state=17)
sgd_logit2 = SGDClassifier(loss='log', random_state=17)

In [ ]:
%%time
logit2.fit(X_train_sparse, y)

In [ ]:
%%time
sgd_logit2.fit(X_train_sparse, y)

In [ ]:
logit_400_users_pred = logit2.predict(X_test_sparse)
sgd_400_users_pred = sgd_logit2.predict(X_test_sparse)

In [ ]:
write_to_submission_file(logit_400_users_pred, 
                         os.path.join(PATH_TO_DATA, 'logit_400_users.csv'))
write_to_submission_file(sgd_400_users_pred, 
                         os.path.join(PATH_TO_DATA, 'sgd_400_users.csv'))

Посмотрим на доли правильных ответов на публичной части (public leaderboard) тестовой выборки этого соревнования.

Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard) для Vowpal Wabbit?

Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard) для SGD?

Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard) для логистической регрессии?

In [ ]:
vw_lb_score, sgd_lb_score, logit_lb_score = [0.18656, 0.18316, 0.19887]

write_answer_to_file(round(vw_lb_score, 3), 'answers/week6/answer6_4.txt')
write_answer_to_file(round(sgd_lb_score, 3), 'answers/week6/answer6_5.txt')
write_answer_to_file(round(logit_lb_score, 3), 'answers/week6/answer6_6.txt')

In [ ]:
В заключение по заданию:

- Про соотношение качества классификации и скорости обучения VW, SGD и logit выводы предлагается сделать самостоятельно
- Пожалуй, задача классификации на 400 классов (идентификация 400 пользователей) решается недостаточно хорошо при честном отделении по времени тестовой выборки от обучающей. Далее мы будем соревноваться в идентификации одного пользователя (Элис) – вот соревнование, в котором предлагается поучаствовать. Не перепутайте!

Удачи!